In [ ]:
import sklearn
import os
import numpy as np
import pandas as pd


# Our evaluation functions¶


In [ ]:
# Upsample the minority class (brakes applied)
from sklearn.utils import resample

def upsample(input_df,col="Brake",majority_value=0):
    '''
    Creates a balanced data set from the dataframe provided to it by upsampling the
    minority class, using col as the column_name of classes to be balanced
    '''    
    #Split by row based on the data class
    df_majority = input_df[input_df[col]==majority_value]
    df_minority = input_df[input_df[col]!=majority_value]
    
    df_minority_upsampled = resample(df_minority,
                                  replace=True,
                                  n_samples=df_majority.shape[0],
                                  random_state=444)
    
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    #print("Before upsampling:\n",df.Brake.value_counts())
    #print("After upsampling:\n",df_upsampled.Brake.value_counts())
    #print(df_upsampled.describe())
    
    return df_upsampled

In [ ]:
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from yellowbrick.classifier.classification_report import ClassificationReport
from yellowbrick.classifier import ConfusionMatrix


def fit_and_evaluate(df_data,model,label):
    """
    Performs K-fold cross validation to create our evaluation scores, and then retrains the model
    on the entire data set. 
    
    df_data: a dataframe of the data to be modeled, with 'y' as the last column
    model: the sklearn model class that we want to create a new instance of
    label: string printed above the output; not stored in any way
    """
    
    #setup output variables
    scores={'precision':[],'recall':[],'accuracy':[], 'f1':[]}
    cm_list=[]
    cm=np.array([[0,0],[0,0]])
    
    #Perform cross validation training
    for train, test in KFold(df_data.shape[0], n_folds=12,shuffle=True):
        df_train, df_test = df_data.iloc[train], df_data.iloc[test]
        
        df_train_upsampled = upsample(df_train,col="Brake",majority_value=0)
        
        X_train, y_train = df_train_upsampled.iloc[:,:-1], df_train_upsampled.iloc[:,-1]
        X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]
        
        estimator = model()
        estimator.fit(X_train, y_train)
        
        expected  = y_test
        predicted = estimator.predict(X_test)
        
        # Append our scores to the tracker
        scores['precision'].append(metrics.precision_score(expected, predicted, average="weighted"))
        scores['recall'].append(metrics.recall_score(expected, predicted, average="weighted"))
        scores['accuracy'].append(metrics.accuracy_score(expected, predicted))
        scores['f1'].append(metrics.f1_score(expected, predicted, average="weighted"))
        
        cm_list.append(confusion_matrix(expected,predicted))
    
    for c in cm_list:
        cm = np.add(cm,cm_list[0])
    
    # Retrain the model on the whole data set
    estimator = model()
    df_train_upsampled = upsample(df_train,col="Brake",majority_value=0)
    estimator.fit(df_train_upsampled.iloc[:,:-1], df_train_upsampled.iloc[:,-1])
    
    print(label)
    print('-----------------')
    print(pd.DataFrame(scores).mean())
    print(cm)
    
    return estimator,scores,cm

In [ ]:
def create_comparison_file(estimator,output_name):
    """
    Takes a fitted model, runs it on our unseen data set, and outputs the resulting predictions to a file.
    """
    test_data_path = 'data/2018-01-31.csv'
    test_df = pd.read_csv(test_data_path, sep=',', header=0)
    test_df = test_df.iloc[:,3:]
    
    test_X = test_df.iloc[:,:-1]
    test_y_actual = test_df.iloc[:,-1]
    
    test_y_predicted = estimator.predict(test_X)
    print(confusion_matrix(test_y_actual,test_y_predicted))
    
    #confusion matrix viz
    visualizer = ConfusionMatrix(estimator, classes=[0,1])
    visualizer.score(test_X, test_y_actual)
    visualizer.poof()
    
    #classification report
    visualizer = ClassificationReport(estimator, classes=['No Brake','Brake'])
    visualizer.score(test_X, test_y_actual)  
    g = visualizer.poof() 
    
    test_comparison_df = pd.DataFrame({'actual':test_y_actual,'predicted':test_y_predicted})
    output_path = 'outputs/' + output_name
    test_comparison_df.to_csv(output_path);

# Get the actual data


In [ ]:
data_path = "data/2018-01-29.csv"

# Get our 3 target columns = accel in each direction plus boolean (1/0) for the class braking/not braking
df = pd.read_csv(data_path, sep=",",header=0)
df = df.iloc[:,3:] #don't use first two columns
df.head()

# Train various models and compare them to a separate dataset¶


In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

classification_models = [KNeighborsClassifier,RandomForestClassifier,SVC]
model_labels = ["KN_Classifier","Random_Forest","SVM"]

for (model,label) in zip(classification_models,model_labels):
    data_path = "data/2018-01-29.csv"
    df = pd.read_csv(data_path, sep=",",header=0)
    df = df.iloc[:,3:] #don't use first two columns
    fitted_model, scores, cm = fit_and_evaluate(df,model,label)
    file_name = label + "_comparison.csv"
    create_comparison_file(fitted_model,file_name)


# New evaluation functions

In [ ]:
from yellowbrick.features import RadViz,Rank2D
from yellowbrick.features.pca import PCADecomposition
from yellowbrick.features.pcoords import ParallelCoordinates

def feature_viz(df,features,labels,classes):
    import warnings
    warnings.filterwarnings('ignore')

    # Extract the numpy arrays from the data frame
    X = df[features].as_matrix()
    y = df[labels].as_matrix()
    
    
    visualizers = [ParallelCoordinates(classes=classes, features=features,normalize='standard', sample=0.1,),
                   RadViz(classes=classes, features=features),
                   Rank2D(features=features, algorithm='pearson'),
                   PCADecomposition(scale=True, center=False, col=y)]
    
    for visualizer in visualizers:
        visualizer.fit(X, y)                # Fit the data to the visualizer
        visualizer.transform(X)             # Transform the data
        visualizer.poof()                   # Draw/show/poof the data




In [ ]:
# Upsample the minority class (brakes applied)
from sklearn.utils import resample

def upsample(input_df,col="Brake",majority_value=0):
    '''
    Creates a balanced data set from the dataframe provided to it by upsampling the
    minority class, using col as the column_name of classes to be balanced
    '''    
    #Split by row based on the data class
    df_majority = input_df[input_df[col]==majority_value]
    df_minority = input_df[input_df[col]!=majority_value]
    
    df_minority_upsampled = resample(df_minority,
                                  replace=True,
                                  n_samples=df_majority.shape[0],
                                  random_state=444)
    
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    #print("Before upsampling:\n",df.Brake.value_counts())
    #print("After upsampling:\n",df_upsampled.Brake.value_counts())
    #print(df_upsampled.describe())
    
    return df_upsampled


In [ ]:
def add_features(df_data):
    from sklearn.preprocessing import PolynomialFeatures
    new_feat = pd.DataFrame()
    new_feat['mag'] = (np.sqrt(df_data['X']**2) + (df_data['Y']**2) +(df_data['Z']**2))
    new_feat['X_diff'] = df_data['X'].diff()
    new_feat['Y_diff'] = df_data['Y'].diff()
    new_feat['Z_diff'] = df_data['Z'].diff()

    for i in range(1,11):
        for axis in ['X','Y','Z']:        
            colname = axis + '_shift' + str(i)
            new_feat[colname] = df_data[axis].shift(i)

    for i in [2,3,4,5,6,7,8,9,10]:
        for axis in ['X','Y','Z']:        
            colname = axis + '_avg' + str(i)
            new_feat[colname] = df_data[axis].rolling(i).mean().shift(1)   


    axes = df_data.iloc[:,:-1]
    poly = PolynomialFeatures(degree=3, interaction_only=True)
    poly_val = pd.DataFrame(poly.fit_transform(axes))
    poly_val.columns = '1','X1','X2','X3','X1X2','X1X3','X2X3','X1X2X3'
    poly_val.drop(['1'],inplace=True,axis=1)
    
    new_feat = pd.merge(new_feat, poly_val, right_index=True, left_index=True)
    df_data = pd.merge(new_feat,df, right_index=True, left_index=True)
    df_data.dropna(inplace=True)
    df_data.reset_index(inplace=True,drop=True)
    
    return df_data

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.feature_selection import SelectFromModel
def select_features(df_data,feature_selection_model):
    '''
    Selects features based on selectfrommodel 
    '''    
    df_data_upsampled = upsample(df_data,col="Brake",majority_value=0)
    model = feature_selection_model()
    sfm = SelectFromModel(model)
    sfm.fit(df_data_upsampled.iloc[:,:-1], df_data_upsampled.iloc[:,-1])
    features = list(df_data_upsampled.iloc[:,:-1].iloc[:,sfm.get_support(indices=True)])
    return features

In [ ]:
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from yellowbrick.classifier.classification_report import ClassificationReport


def fit_and_evaluate_new(df_data,feature_selection_model,model,label):
    """
    Performs K-fold cross validation to create our evaluation scores, and then retrains the model
    on the entire data set. 
    
    df_data: a dataframe of the data to be modeled, with 'y' as the last column
    model: the sklearn model class that we want to create a new instance of
    label: string printed above the output; not stored in any way
    """
    
    #setup output variables
    scores={'precision':[],'recall':[],'accuracy':[], 'f1':[]}
    cm_list=[]
    cm=np.array([[0,0],[0,0]])
    
    #add more features
    df_data = add_features(df_data)
    new_columns = select_features(df_data,feature_selection_model)
    
    new_columns.append('Brake')
    df_data = df_data[new_columns]

    
    #Perform cross validation training
    for train, test in KFold(df_data.shape[0], n_folds=12,shuffle=True):
        df_train, df_test = df_data.iloc[train], df_data.iloc[test]
        
        
        df_train_upsampled = upsample(df_train,col="Brake",majority_value=0)
        
        X_train, y_train = df_train_upsampled.iloc[:,:-1], df_train_upsampled.iloc[:,-1]
        X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]
        
        estimator = model()
        estimator.fit(X_train, y_train)
        
        expected  = y_test
        predicted = estimator.predict(X_test)
        
        # Append our scores to the tracker
        scores['precision'].append(metrics.precision_score(expected, predicted, average="weighted"))
        scores['recall'].append(metrics.recall_score(expected, predicted, average="weighted"))
        scores['accuracy'].append(metrics.accuracy_score(expected, predicted))
        scores['f1'].append(metrics.f1_score(expected, predicted, average="weighted"))
        
        cm_list.append(confusion_matrix(expected,predicted))
        

    
    for c in cm_list:
        cm = np.add(cm,cm_list[0])
        
    
    # Retrain the model on the whole data set df_train_upsampled was here - changed
    estimator = model()
    df_data_upsampled = upsample(df_data,col="Brake",majority_value=0)
    estimator.fit(df_data_upsampled.iloc[:,:-1], df_data_upsampled.iloc[:,-1])
                                      
    print(label)
    print('-----------------')
    print(pd.DataFrame(scores).mean())
    print(cm)
    print(new_columns)

    
    return estimator,scores,cm,new_columns


In [ ]:
def create_comparison_file_new(estimator,output_name,columns):
    """
    Takes a fitted model, runs it on our unseen data set, and outputs the resulting predictions to a file.
    """
    test_data_path = 'data/2018-01-31.csv'
    test_df = pd.read_csv(test_data_path, sep=',', header=0)
    test_df = test_df.iloc[:,3:]
    
    test_df = add_features(test_df)
    test_df = test_df[columns]
    test_X = test_df.iloc[:,:-1]
    test_y_actual = test_df.iloc[:,-1]
    
    test_y_predicted = estimator.predict(test_X)
    print(confusion_matrix(test_y_actual,test_y_predicted))
    
    #confusion matrix viz
    visualizer = ConfusionMatrix(estimator, classes=[0,1])
    visualizer.score(test_X, test_y_actual)
    visualizer.poof()
    
    #classification report
    visualizer = ClassificationReport(estimator, classes=['No Brake','Brake'])
    visualizer.score(test_X, test_y_actual)  
    g = visualizer.poof() 

    test_comparison_df = pd.DataFrame({'actual':test_y_actual,'predicted':test_y_predicted})
    output_path = 'outputs/' + output_name
    test_comparison_df.to_csv(output_path);


# Get the actual data


In [ ]:
data_path = "data/2018-01-29.csv"

# Get our 3 target columns = accel in each direction plus boolean (1/0) for the class braking/not braking
df = pd.read_csv(data_path, sep=",",header=0)
df = df.iloc[:,3:] #don't use first two columns
df.head()

#  Feature Analysis

In [ ]:
features = ["X","Y","Z"]
classes = ["no brake","brake"]
labels = "Brake"
feature_viz(df,features,labels,classes)

adding more features

In [ ]:
data_path = "data/2018-01-29.csv"

# Get our 3 target columns = accel in each direction plus boolean (1/0) for the class braking/not braking
df = pd.read_csv(data_path, sep=",",header=0)
df = df.iloc[:,3:] #don't use first two columns
df.head()

df = add_features(df)
df.columns.values


In [ ]:
df = df[[ 'X_diff', 'Y_diff', 'Z_diff','X','Y','Z','Brake']]
features = [ 'X_diff', 'Y_diff', 'Z_diff','X','Y','Z']
classes = ["no brake","brake"]
labels = "Brake"
feature_viz(df,features,labels,classes)

# Train various models and compare them to a separate dataset

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

feature_selection_models = [Lasso,Ridge,ElasticNet]
feature_selection_labels = ["Lasso","Ridge","ElasticNet"]
classification_models = [KNeighborsClassifier,RandomForestClassifier,SVC]
model_labels = ["KN_Classifier","Random_Forest","SVM"]

for (feature_selection_model,feature_selection_label) in zip(feature_selection_models,feature_selection_labels):
    print (feature_selection_label)
    print('-----------------\n\n')
    for (model,label) in zip(classification_models,model_labels):
        data_path = "data/2018-01-29.csv"
        df = pd.read_csv(data_path, sep=",",header=0)
        df = df.iloc[:,3:] #don't use first two columns
        fitted_model, scores, cm, new_columns = fit_and_evaluate_new(df,feature_selection_model,model,label)
        file_name = feature_selection_label + label + "comparison.csv"
        create_comparison_file_new(fitted_model,file_name,new_columns)
    